In [89]:
import requests
import json

Set the drug that we want to find associations for

In [90]:
myDrug = "INFLIXIMAB"

Make the API call to find all the drugs

In [91]:
size = 10000
publicURL = 'https://www.targetvalidation.org/'
biogenURL = 'http://localhost:8000/'
link = publicURL + 'api/latest/public/evidence/filter?datatype=known_drug&fields=disease.efo_info.therapeutic_area.labels&fields=drug.molecule_name&fields=target.gene_info.symbol&size='
url = link + str(size)

response = requests.get(url)
output = response.json()

#print out the json for debug
#print json.dumps(output, indent=2)

Collect information about diseases and targets associated with our drug

In [92]:
total = output['total']
fromCount = size - 1
diseases = {}
targets = {}

while (fromCount < total):
    
    if('data' in output):
        for data in output['data']:
        
            if(myDrug.lower() == data['drug']['molecule_name'].lower()):
                for x in data['disease']['efo_info']['therapeutic_area']['labels']:
                    diseases[x] = 'true'
                
                targets[data['target']['gene_info']['symbol']] = 'true'
                #print "Drug: " + data['drug']['molecule_name']
                #print ', '.join([str(x) for x in data['disease']['efo_info']['therapeutic_area']['labels']])
                #print "Target: " + data['target']['gene_info']['symbol']
    
    fromCount = fromCount + size
    url = link + str(size) + "&from=" + str(fromCount)

    response = requests.get(url)
    output = response.json()
    #print json.dumps(output, indent=2)
    
print "DRUG: " + myDrug

print "\nDISEASES: "

for key in diseases.keys():
    print "\t" + key
    
print "\nTARGETS:"

for key in targets.keys():
    print "\t" + key

DRUG: INFLIXIMAB

DISEASES: 
	eye disease
	skeletal system disease
	immune system disease
	skin disease
	cardiovascular disease
	nervous system disease
	digestive system disease

TARGETS:
	TNF
